In [123]:
import numpy as np
import pandas as pd
import pylab as pl
import os
import sqlalchemy as sql
from pandas import Panel
from pandas.stats import plm
import geopandas as gpd
from datetime import datetime
from pandas.stats import plm
import statsmodels.formula.api as smf

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [124]:
df = pd.read_csv('by_hour_day.csv', names=['day','count','cam'])
df = df[df['day'] != 'Error']
df.head(3)

,day,count,cam
0,2017-06-27 20,19,121
1,2017-06-27 20,48,125
2,2017-06-27 20,9,126


In [125]:
df['date'] = pd.to_datetime(df['day'] + str(':00:00'), format='%Y-%m-%d %H:%M:%S')
df.set_index(df['date'], inplace = True)

In [126]:
df.head(3)

,day,count,cam,date
date,,,,
2017-06-27 20:00:00,2017-06-27 20,19,121,2017-06-27 20:00:00
2017-06-27 20:00:00,2017-06-27 20,48,125,2017-06-27 20:00:00
2017-06-27 20:00:00,2017-06-27 20,9,126,2017-06-27 20:00:00


In [127]:
# get day of week dummies
# hrs = ['0h','1h','2h', '3h', '4h', '5h', '6h', '7h', '8h', '9h', '10h', '11h', '12h', '13h', '14h', '15h', '16h',\
#        '17h', '18h', '19h',\
#        '20h', '21h', '22h', '23h']
# for i, hr in enumerate(hrs):
#     df[hr] = (df.index.hour == i)*1

intervals = ['6-10', '10-14', '14-18', '18-22', '22-2', '2-6']

df['6-10'] = ((df.index.hour > 5) & (df.index.hour < 10))*1

df['10-14'] = ((df.index.hour > 9) & (df.index.hour < 14))*1

df['14-18'] = ((df.index.hour > 13) & (df.index.hour < 18))*1

df['18-22'] = ((df.index.hour > 17) & (df.index.hour < 22))*1

df['22-2'] = ((df.index.hour > 21) & (df.index.hour < 2))*1

df['2-6'] = ((df.index.hour > 1) & (df.index.hour < 6))*1


# get month dummies
days = ['weekday','weekend']
df['weekday'] = (df.index.dayofweek < 5)*1
df['weekend'] = (df.index.dayofweek > 4)*1

In [128]:
df.head(3)

,day,count,cam,date,6-10,10-14,14-18,18-22,22-2,2-6,weekday,weekend
date,,,,,,,,,,,,
2017-06-27 20:00:00,2017-06-27 20,19,121,2017-06-27 20:00:00,0,0,0,1,0,0,1,0
2017-06-27 20:00:00,2017-06-27 20,48,125,2017-06-27 20:00:00,0,0,0,1,0,0,1,0
2017-06-27 20:00:00,2017-06-27 20,9,126,2017-06-27 20:00:00,0,0,0,1,0,0,1,0


In [129]:
df = df[df.cam.isin([504, 540, 398, 794, 503, 838, 163, 491, 488, 91])]
# df = df[df.cam.isin([504])]

In [130]:
df.cam.unique()

array([163, 398, 488, 491, 503, 504, 540, 794, 838])

In [131]:
df.shape

(2871, 12)

In [132]:
df_panel = df[df.weekend == 1].set_index(['date','cam'])
df_panel['count_lag4'] = df_panel['count'].shift(4)
df_panel['count_lag24'] = df_panel['count'].shift(24)

In [122]:
plm.PanelOLS(y = df_panel['count'], x = df_panel[['count_lag4','6-10', '10-14', '14-18', '18-22', '22-2', '2-6',\
                                                  ]],\
                                                    entity_effects=True)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <count_lag4> + <6-10> + <10-14> + <14-18> + <18-22> + <22-2>
             + <2-6> + <FE_398> + <FE_488> + <FE_491> + <FE_503> + <FE_504> + <FE_540>
             + <FE_794> + <FE_838> + <intercept>

Number of Observations:         1886
Number of Degrees of Freedom:   15

R-squared:         0.6914
Adj R-squared:     0.6891

Rmse:             71.0330

F-stat (15, 1871):   299.4748, p-value:     0.0000

Degrees of Freedom: model 14, resid 1871

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
    count_lag4     0.4204     0.0210      20.02     0.0000     0.3793     0.4616
          6-10    69.9516     5.8697      11.92     0.0000    58.4469    81.4563
         10-14    85.6579     6.3259      13.54     0.

In [133]:
plm.PanelOLS(y = df_panel['count'], x = df_panel[['count_lag4','6-10', '10-14', '14-18', '18-22', '22-2', '2-6',\
                                                  ]],\
                                                    entity_effects=True)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <count_lag4> + <6-10> + <10-14> + <14-18> + <18-22> + <22-2>
             + <2-6> + <FE_398> + <FE_488> + <FE_491> + <FE_503> + <FE_504> + <FE_540>
             + <FE_794> + <FE_838> + <intercept>

Number of Observations:         860
Number of Degrees of Freedom:   15

R-squared:         0.6971
Adj R-squared:     0.6921

Rmse:             58.7626

F-stat (15, 845):   138.9311, p-value:     0.0000

Degrees of Freedom: model 14, resid 845

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
    count_lag4     0.4231     0.0321      13.20     0.0000     0.3603     0.4859
          6-10    31.4025     7.0342       4.46     0.0000    17.6155    45.1896
         10-14    79.0074     8.1217       9.73     0.000

In [84]:
plm.PanelOLS(y = df_panel['count'], x = df_panel[['count_lag24','weekday','weekend']],\
                 entity_effects=True)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <count_lag24> + <weekday> + <weekend> + <FE_398> + <FE_488>
             + <FE_491> + <FE_503> + <FE_504> + <FE_540> + <FE_794> + <FE_838>
             + <intercept>

Number of Observations:         2730
Number of Degrees of Freedom:   11

R-squared:         0.3481
Adj R-squared:     0.3457

Rmse:             98.3380

F-stat (11, 2719):   145.1937, p-value:     0.0000

Degrees of Freedom: model 10, resid 2719

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
   count_lag24     0.3037     0.0183      16.58     0.0000     0.2678     0.3396
       weekday    68.8344     2.6065      26.41     0.0000    63.7256    73.9432
       weekend    55.7469     2.9497      18.90     0.0000    49.9655    61.5283
    

In [53]:
plm.PanelOLS(y = df_panel['count'], x = df_panel[['count_lag24','0h', '1h', '2h', '3h', '4h',\
       '5h', '6h', '7h', '8h', '9h', '10h', '11h', '12h','13h',\
       '14h','15h','16h','17h','18h','19h','20h','21h','22h',\
       '23h']], \
                 entity_effects=True)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <count_lag24> + <0h> + <1h> + <2h> + <3h> + <4h> + <5h> + <6h>
             + <7h> + <8h> + <9h> + <10h> + <11h> + <12h> + <13h> + <14h> + <15h>
             + <16h> + <17h> + <18h> + <19h> + <20h> + <21h> + <22h> + <23h>
             + <FE_398> + <FE_488> + <FE_491> + <FE_503> + <FE_504> + <FE_540>
             + <FE_794> + <FE_838> + <intercept>

Number of Observations:         2730
Number of Degrees of Freedom:   33

R-squared:         0.6801
Adj R-squared:     0.6763

Rmse:             69.1713

F-stat (33, 2697):   179.1545, p-value:     0.0000

Degrees of Freedom: model 32, resid 2697

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
   count_lag24    -0.1649     0.0187      -8.83     0.0000    -

In [265]:
md = smf.mixedlm("count ~ weekday + weekkend", df, groups=df["cam"]).fit()
md.summary()

/opt/rh/anaconda/root/lib/python2.7/site-packages/statsmodels/regression/mixed_linear_model.py:1705: ConvergenceWarning: Gradient optimization failed.
  warnings.warn(msg, ConvergenceWarning)
/opt/rh/anaconda/root/lib/python2.7/site-packages/statsmodels/base/model.py:971: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
============================================================================
Model:                   MixedLM       Dependent Variable:       count      
No. Observations:        2754          Method:                   REML       
No. Groups:              9             Scale:                    10584.7163 
Min. group size:         306           Likelihood:               -16667.3619
Max. group size:         306           Converged:                No         
Mean group size:         306.0                                              
----------------------------------------------------------------------------
               Coef.     Std.Err.     z   P>|z|     [0.025         0.975]   
----------------------------------------------------------------------------
Intercept       73.404 55972661.820 0.000 1.000 -109704327.882 109704474.690
weekday         45.571 55972661.820 0.000 1.000 -109704355.715 109704446.857
weekkend        27.833 55972661.820 0.000 1.000 -109704373.453 109704429.119
Intercept RE 10584.716                                                      
============================================================================

"""

In [271]:
df.columns = ['day','count','cam','date' ,'0h', '1h', '2h', '3h', '4h',\
       '5h', '6h', '7h', '8h', '9h', '10h', '11h', '12h','13h',\
       '14h','15h','16h','17h','18h','19h','20h','21h','22h',\
       '23h', 'weekday','weekend']

In [270]:
df.columns

Index([u'day', u'count', u'cam', u'date', u'0h', u'1h', u'2h', u'3h', u'4h',
       u'5h', u'6h', u'7h', u'8h', u'9h', u'10h', u'11h', u'12h', u'13h',
       u'14h', u'15h', u'16h', u'17h', u'18h', u'19h', u'20h', u'21h', u'22h',
       u'23h', u'weekday', u'weekkend'],
      dtype='object')

In [1]:
md.sumary()

NameError: name 'md' is not defined